# Find Similar Compounds
Given our list of "interesting" compounds, see if we can find any similar stable compounds in the OQMD

In [1]:
from pymatgen import Composition
import pandas as pd
import json

## Load in Stable Compounds from OQMD
Reading from the datafile that was used to generate the training set for the DL model.

In [2]:
oqmd_all = pd.read_csv('oqmd_all.txt', delim_whitespace=True)
print('Read %d entries'%len(oqmd_all))

Read 506114 entries


C:\Users\ward6\AppData\Local\Continuum\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
oqmd_all['stability'] = pd.to_numeric(oqmd_all['stability'], 'coerce')

In [4]:
oqmd_all.query('stability <= 0', inplace=True)
print('%d stable compounds'%len(oqmd_all))

21947 stable compounds


### Generate Lookup Values for Each Entry
Classify each entry by the stoichiometry and group of each element. Examples:
- NaCl is 50% of a group 1 element and 50% of group 17
- NaKCl2 is 25% of two different group 1 elements and 50% of a group 17 element

In [5]:
oqmd_all['comp_obj'] = [Composition(x) for x in oqmd_all['comp']]

Compute lookup values

In [6]:
def get_prototype(comp):
    return tuple(sorted((e.group, y) for e,y in comp.fractional_composition.items())) 

In [7]:
oqmd_all['prototype'] = oqmd_all['comp_obj'].apply(get_prototype)

Get list of examples for each prototype

In [8]:
prototypes = dict([(x,[c.get_integer_formula_and_factor()[0] for c in group['comp_obj']]) 
                   for x,group in oqmd_all.groupby('prototype')])

C:\Users\ward6\AppData\Local\Continuum\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:398: UserWarning: No electronegativity for Ne. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Users\ward6\AppData\Local\Continuum\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:398: UserWarning: No electronegativity for Ar. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Users\ward6\AppData\Local\Continuum\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:398: UserWarning: No electronegativity for He. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)


In [9]:
print('Found %d prototypes'%len(prototypes))

Found 9211 prototypes


## Find if Interesting Compositions are Similar to those in the OQMD
Use the prototype list we worked up earlier

In [10]:
interesting_list = json.load(open('interesting_compounds.list'))

In [11]:
interesting_list = pd.DataFrame({'composition': interesting_list})

In [12]:
interesting_list['comp_obj'] = [Composition(x) for x in interesting_list['composition']]

In [13]:
interesting_list['prototype'] = interesting_list['comp_obj'].apply(get_prototype)

In [14]:
interesting_list['similiar'] = [prototypes.get(x,[]) for x in interesting_list['prototype']]

The following table shows similar compounds to those from our DL predictions. Each example "similar" structure is a stable compound in the OQMD

In [15]:
interesting_list

composition          comp_obj  \
0      KSc2Br7       (K, Sc, Br)   
1       KHfBr5       (K, Hf, Br)   
2    CsNa2CdF4   (Cs, Na, Cd, F)   
3    Na2CrPbF5   (Na, Cr, Pb, F)   
4       K2W2N5         (K, W, N)   
5      LiTi4N5       (Li, Ti, N)   
6    Ba3NaPtO4   (Ba, Na, Pt, O)   
7    K2P(WN2)2      (K, P, W, N)   
8     Sc2SeBr5      (Sc, Se, Br)   
9      Sc3SBr6       (Sc, S, Br)   
10   Sr3Cu2IO4    (Sr, Cu, I, O)   
11    Zr6RhIO2    (Zr, Rh, I, O)   
12     Hf2Br6O       (Hf, Br, O)   
13     Sc3Br6O       (Sc, Br, O)   
14   Sr3Cu2IO4    (Sr, Cu, I, O)   
15    Zr6RhIO2    (Zr, Rh, I, O)   
16      K2OsO5        (K, Os, O)   
17      AgRuO3       (Ag, Ru, O)   
18     YAlV2O6     (Y, Al, V, O)   
19    TiMnSnO5   (Ti, Mn, Sn, O)   
20      TiCrO3       (Ti, Cr, O)   
21      TiMnO3       (Ti, Mn, O)   
22   Ti2MnCrO6   (Ti, Mn, Cr, O)   
23   ScTiCr2O6   (Sc, Ti, Cr, O)   
24    HfAl5Ir3      (Hf, Al, Ir)   
25     YAl4Ir3       (Y, Al, Ir)   
26  Sc5NiSn3Mo  (Sc, Ni, Sn, Mo)   
27   ZrAl5OsRh  (Zr, Al, Os, Rh)   
28    TiAl5Rh4      (Ti, Al, Rh)   
29    ZrAl5Rh4      (Zr, Al, Rh)   
30  TiAl5NiRh3  (Ti, Al, Ni, Rh)   
31  Al6CrCoRh2  (Al, Cr, Co, Rh)   
32   Ti2In3Ni4      (Ti, In, Ni)   
33  TiAl5NiRh3  (Ti, Al, Ni, Rh)   
34  Al6CrCoRh2  (Al, Cr, Co, Rh)   

                                            prototype  \
0                     ((1, 0.1), (3, 0.2), (17, 0.7))   
1   ((1, 0.14285714285714285), (4, 0.1428571428571...   
2     ((1, 0.125), (1, 0.25), (12, 0.125), (17, 0.5))   
3   ((1, 0.2222222222222222), (6, 0.11111111111111...   
4   ((1, 0.2222222222222222), (6, 0.22222222222222...   
5                     ((1, 0.1), (4, 0.4), (15, 0.5))   
6   ((1, 0.1111111111111111), (2, 0.33333333333333...   
7   ((1, 0.2222222222222222), (6, 0.22222222222222...   
8               ((3, 0.25), (16, 0.125), (17, 0.625))   
9                    ((3, 0.3), (16, 0.1), (17, 0.6))   
10        ((2, 0.3), (11, 0.2), (16, 0.4), (17, 0.1))   
11         ((4, 0.6), (9, 0.1), (16, 0.2), (17, 0.1))   
12  ((4, 0.2222222222222222), (16, 0.1111111111111...   
13                   ((3, 0.3), (16, 0.1), (17, 0.6))   
14        ((2, 0.3), (11, 0.2), (16, 0.4), (17, 0.1))   
15         ((4, 0.6), (9, 0.1), (16, 0.2), (17, 0.1))   
16               ((1, 0.25), (8, 0.125), (16, 0.625))   
17                   ((8, 0.2), (11, 0.2), (16, 0.6))   
18         ((3, 0.1), (5, 0.2), (13, 0.1), (16, 0.6))   
19  ((4, 0.125), (7, 0.125), (14, 0.125), (16, 0.6...   
20                    ((4, 0.2), (6, 0.2), (16, 0.6))   
21                    ((4, 0.2), (7, 0.2), (16, 0.6))   
22          ((4, 0.2), (6, 0.1), (7, 0.1), (16, 0.6))   
23          ((3, 0.1), (4, 0.1), (6, 0.2), (16, 0.6))   
24  ((4, 0.1111111111111111), (9, 0.33333333333333...   
25                ((3, 0.125), (9, 0.375), (13, 0.5))   
26         ((3, 0.5), (6, 0.1), (10, 0.1), (14, 0.3))   
27  ((4, 0.125), (8, 0.125), (9, 0.125), (13, 0.625))   
28                    ((4, 0.1), (9, 0.4), (13, 0.5))   
29                    ((4, 0.1), (9, 0.4), (13, 0.5))   
30         ((4, 0.1), (9, 0.3), (10, 0.1), (13, 0.5))   
31          ((6, 0.1), (9, 0.1), (9, 0.2), (13, 0.6))   
32  ((4, 0.2222222222222222), (10, 0.4444444444444...   
33         ((4, 0.1), (9, 0.3), (10, 0.1), (13, 0.5))   
34          ((6, 0.1), (9, 0.1), (9, 0.2), (13, 0.6))   

                     similiar  
0                    [KSc2F7]  
1                    [RbHfF5]  
2                          []  
3                          []  
4                          []  
5                          []  
6                          []  
7                          []  
8                          []  
9                          []  
10                         []  
11                         []  
12                         []  
13                         []  
14                         []  
15                         []  
16                         []  
17                         []  
18                         []  
19           